In [1]:
import re

def normalize_value(text):
    try:
        # Definir patrones de expresiones regulares para los dos casos
        pattern1 = re.compile(r'([\d.]+)to([\d.]+)')
        pattern2 = re.compile(r'([\d.]+)toover([\d.]+)')
        pattern3 = re.compile(r'([\d.]+)-([\d.]+)')
        pattern4 = re.compile(r'([\d.]+)-([\d.]+)')
        pattern5 = re.compile(r'([\d.]+)and([\d.]+)')
        pattern6 = re.compile(r'([\d.]+)--([\d.]+)')
        pattern7 = re.compile(r'([\d.]+)-to ([\d.]+)-')
        pattern8 = re.compile(r'([\d.]+)[×xX]10\((-?\d+)\)')
        pattern9 = re.compile(r'(\d+)\.(\d+)\.(\d+)')
        pattern10 = re.compile(r'([\d.]+)[×xX]10\((-?\d+)\)')
        pattern11 = re.compile(r'([\d.]+)approximately([\d.]+)')
        # Intentar hacer coincidir con el primer patrón
        match1 = pattern1.search(text)
        if match1:
            number1 = float(match1.group(1))
            number2 = float(match1.group(2))
            return (number1 + number2) / 2

        # Si no coincide con el primer patrón, intentar con el segundo patrón
        match2 = pattern2.search(text)
        if match2:
            number1 = float(match2.group(1))
            number2 = float(match2.group(2))
            return (number1 + number2) / 2
        
        match3 = pattern3.search(text)
        if match3:
            number1 = float(match3.group(1))
            number2 = float(match3.group(2))
            return (number1 + number2) / 2

        match4 = pattern4.search(text)
        if match4:
            number1 = float(match4.group(1))
            number2 = float(match4.group(2))
            return (number1 + number2) / 2
        
        match5 = pattern5.search(text)
        if match5:
            number1 = float(match5.group(1))
            number2 = float(match5.group(2))
            return (number1 + number2) / 2
        
        match6 = pattern6.search(text)
        if match6:
            number1 = float(match6.group(1))
            number2 = float(match6.group(2))
            return (number1 + number2) / 2
        
        match7 = pattern7.search(text)
        if match7:
            number1 = float(match7.group(1))
            number2 = float(match7.group(2))
            return (number1 + number2) / 2
        
        match8 = pattern8.search(text)
        if match8:
            base = float(match8.group(1))
            exponent = int(match8.group(2))
            return base * (10 ** exponent)
        
        match9 = pattern9.search(text)
        if match9:
            number1 = match9.group(1)
            number2 = match9.group(2)
            number3 = match9.group(3)
            return float(number1 + number2 + '.' + number3)
        
        match10 = pattern10.search(text)
        if match10:
            base = float(match10.group(1))
            exponent = int(match10.group(2))
            return base * (10 ** exponent)
        
        match11 = pattern11.search(text)
        if match11:
            number1 = float(match11.group(1))
            number2 = float(match11.group(2))
            return (number1 + number2) / 2
        
    except:
        pass
    return text  # No se pudo encontrar un patrón válido

In [2]:
from word2number import w2n
def convert_word_to_number(word):
    try:
        return str(w2n.word_to_num(word))
    except ValueError:
        return word

In [3]:
import sqlite3
import pandas as pd
import numpy as np
file=open("log.txt", "w")
# Conectar a la base de datos
conn = sqlite3.connect('DRUG_DB.db')
cursor = conn.cursor()

# Obtener los nombres de las tablas (drogas)
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
table_names = cursor.fetchall()
cont=0
# Iterar a través de las tablas
for table_name in table_names:
    table_name = table_name[0]
    
    # Agregar la columna "accepted_value" a la tabla si aún no existe
    add_column_query = f"""ALTER TABLE "{table_name}" ADD COLUMN accepted_value BOOLEAN DEFAULT 1"""
    cursor.execute(add_column_query)
    conn.commit()


    # Obtener los parámetros únicos en esta tabla
    params_query = f"""SELECT DISTINCT Parameter FROM "{table_name}" """
    unique_params = [row[0] for row in cursor.execute(params_query).fetchall()]
    
    # Iterar a través de los parámetros únicos
    for param in unique_params:
        # Obtener los valores asociados a este parámetro en la tabla
        values_query = f"""SELECT Value FROM "{table_name}" WHERE Parameter = "{param}" """
        values = [row[0] for row in cursor.execute(values_query).fetchall()]

        # Filtrar los valores None del arreglo
        filtered_values = [value for value in values if value is not None]

       
        if len(filtered_values) > 0:
            numeric_values = []
            
            for value in filtered_values:
                original_value=value
                # Normalizaciones
                value=value.replace(',', '.')
                value=value.replace('∼','')
                value = ' '.join([convert_word_to_number(word) for word in value.split()])
                value=value.replace(' ','')
                
                
                value=normalize_value(value)
    
                try:
                    value=float(value)
                    update_query = f"""UPDATE "{table_name}" SET Value = ? WHERE Parameter = ? AND Value = ?"""
                    cursor.execute(update_query, (str(value), param, str(original_value)))
                    # Guardar los cambios en la base de datos

                    
                except:
                    cont=cont+1
                    file.write(f'{value}\n')
                    print(value)
                    cursor.execute(f"""UPDATE "{table_name}" SET accepted_value = ? WHERE Parameter = ? AND Value = ?""",
                                  (0, param, str(original_value)))
   
            conn.commit()
# Cerrar la conexión a la base de datos
conn.close()
file.close()
print(cont)

OperationalError: duplicate column name: accepted_value

In [4]:
# Definir el diccionario de conversiones de unidades por parámetro
unit_conversions = {
    "halflife": {
        "h": 1,   
        "min":0.0166667,  
        
    },
   
    "cmax": {
        "ng/ml": 1,
    "μg/ml": 1000,
    "mg/l": 1000,
    "μg/l": 1,
    "pg/ml": 1000,
    "mg/ml": 1000000,
    "mcg/ml": 1000,  
    "ng/l": 0.001,     
    },
     "clearance": {
    "ml/kg·min": 1,
    "l/h·kg": 16.66,
    "ml/h·kg": 0.0166,
    "l/kg·min": 1000,
    "ml/d·kg": 0.0007,
    "min·ml/kg": 1, 
    "h·l/kg": 60000,
    "l/d·kg": 0.6944444444,
    "ml/·kg·min": 1, 
    "ml/kg·minute": 1,
    "l/kg·mn": 1000,
},
"bioavailability":{
    "%":1,
},
"maximaltolerateddose":{
  "mg": 1,
},
"auc": 
{
  "h·ng/ml": 1,
  "h·μg/ml": 1000,
  "h·mg/l": 1000,
  "ng/h·ml": 1,                 
  "μg/h·ml":   1000,                
  "h·μg/l": 1,
  "min·μg/ml": 16.67,
  "mg/h·l":  1000,                 
  "h·pg/ml": 0.001,
  "μg/h·l":  1,                 
  "mg/min·ml":  16666.67, 
  "mg·min/ml": 16666.67, 
        
    "min·ng/ml": 0.01667,          

    "μg/min·ml": 16.67  ,          
    "ngh/ml": 1,                
    "ng/min·ml":   0.01667 ,
    "μgh/ml": 1000, 
    "mg·min/l": 16.67,         

    "d·ng/ml": 24,              
           

    "ml·ng/h":  1,               

                
    "pg/h·ml":  0.001,
    "hng/ml": 1,            
    "mg/l·min": 16.67,         
    "h·mg/ml": 1000000,                  
    "ng-h/ml": 1,               
    "min·μg/l": 0.0167,
    "d·μg/ml": 24000,
    "ng/d·ml": 24,
    "h·ng/l": 0.001,
    "mgh/l": 100,
    "h·ug/ml": 1000,
    "ng/h·l":   0.001,
    "ng/mlh": 1,    
    "h·mcg/ml": 1000,
    "l/h·kg": 1000000000,
    "min·pg/ml": 0.0000167, 
    "μg-hr/ml": 1000,
},
"timetocmax": {
    "h":1,
    "min":0.0167,
    "d":24
},
"cssmax":{
"ng/ml": 1,
    "μg/ml": 1000,
    "mg/l": 1000,
    "μg/l": 1,
 
}



}
# def replace_mu_with_micro(input_string):
#     return input_string.replace('μ', '\u03BC').replace('\u00b5', '\u03BC')

# unit_conversions = {
#     replace_mu_with_micro(key): value
#     for key, value in original_unit_conversions.items()
# }

# print(unit_conversions)



In [5]:
import sqlite3
import re


# Conectar a la base de datos SQLite
db_file = "DRUG_DB.db"  # Reemplaza con el nombre de tu archivo de base de datos
conn = sqlite3.connect(db_file)
cursor = conn.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = [table[0] for table in cursor.fetchall()]
# Recorrer todas las tablas y actualizar los valores numéricos de las instancias
for table_name in tables:
     # añadimos columna 'accepted_unit'
    try:
        add_column_query = f"""ALTER TABLE "{table_name}" ADD COLUMN accepted_unit BOOLEAN DEFAULT 1"""
        cursor.execute(add_column_query)
    except:
        pass 

    cursor.execute(f"""SELECT Parameter, Value, Units FROM "{table_name}" WHERE Accepted_value = 1;""")
    for row in cursor.fetchall():
        parameter = row[0]
        value = row[1]
        units = row[2]
       
        # units_normalized = units.replace('μ', '\u03BC').replace('\u00b5', '\u03BC')
        # units_cleaned = re.sub(r'\s+', '', units_normalized)
        if parameter in unit_conversions and units in unit_conversions[parameter]:
            conversion_factor = unit_conversions[parameter][units]
            converted_value = float(value) * float(conversion_factor)
            
            # sacar unidad estandar
            first_unit = list(unit_conversions[parameter].keys())[0]
            #print("cambio",units,first_unit, parameter)

            # Actualizar el valor en la base de datos con el valor convertido
            update_query = f"""UPDATE "{table_name}" SET Value = ?, Units = ? WHERE Parameter = ? AND Units = ? AND Value = ?"""
            cursor.execute(update_query, (converted_value, first_unit, parameter, units, value))
            
            
        else:
            cursor.execute(f"""UPDATE "{table_name}" SET accepted_unit = ? WHERE Parameter = ? AND Value = ? AND Units = ?""",
                                   (0, parameter, value, units))
            #print("NO Lo cojo: ",units_cleaned,units, parameter," --- ",list(unit_conversions[parameter].keys())[0])
            

        

# Commit y cerrar la conexión a la base de datos
conn.commit()
conn.close()

print("Valores convertidos y actualizados correctamente.")


Valores convertidos y actualizados correctamente.
